# Neural Network Approach


# Import libraries & Load Dataset

In [1]:
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import f1_score, roc_auc_score, confusion_matrix
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
import xgboost as xgb

In [2]:
df = pd.read_csv("./Dataset/processed_train.csv")
x = df.copy()
x = x.drop(columns='stroke')
y = df['stroke']
smote = SMOTE()
x_smote, y_smote = smote.fit_resample(x, y)
x_train, x_test, y_train, y_test = train_test_split(x_smote, y_smote, test_size= 0.2, random_state= 42)

## NN Classifier

In [3]:
from keras.models import Sequential
from keras.layers import Dense

In [4]:
# define the keras model
model = Sequential()
model.add(Dense(12, input_dim=25, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 12)                312       
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 104       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 9         
Total params: 425
Trainable params: 425
Non-trainable params: 0
_________________________________________________________________


### Training

In [6]:
model.fit(x_train, y_train, epochs=150, batch_size=32)

Epoch 1/150
190/190 [==============================] - 0s 757us/step - loss: 0.4944 - accuracy: 0.7906
Epoch 2/150
190/190 [==============================] - 0s 737us/step - loss: 0.2309 - accuracy: 0.9182
Epoch 3/150
190/190 [==============================] - 0s 747us/step - loss: 0.1707 - accuracy: 0.9375
Epoch 4/150
190/190 [==============================] - 0s 786us/step - loss: 0.1468 - accuracy: 0.9460
Epoch 5/150
190/190 [==============================] - 0s 742us/step - loss: 0.1303 - accuracy: 0.9531
Epoch 6/150
190/190 [==============================] - 0s 721us/step - loss: 0.1170 - accuracy: 0.9577
Epoch 7/150
190/190 [==============================] - 0s 716us/step - loss: 0.1075 - accuracy: 0.9613
Epoch 8/150
190/190 [==============================] - 0s 747us/step - loss: 0.0991 - accuracy: 0.9650
Epoch 9/150
190/190 [==============================] - 0s 758us/step - loss: 0.0917 - accuracy: 0.9666
Epoch 10/150
190/190 [==============================] - 0s 742us/step - l

### Prediction

In [7]:
y_pred = model.predict_classes(x_test)
y_pred

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


array([[1],
       [0],
       [0],
       ...,
       [0],
       [1],
       [0]])

In [8]:
f1_score(y_test,y_pred)

0.9792085848423878

## Generate Submission

In [40]:
def generate_submission(model, test_data, model_name = ""): 
    ''' 
    Generate submission based on given model and test set 
    param: 
        model: model must have predict method 
        test_data: dataframe 
        model_name  
    return: 
        None
    '''  
    test_data.drop(columns= 'Unnamed: 0', inplace=True)
    x = test_data.copy()
    x.drop(columns='id', inplace=True)
    id_test = test_data['id']
    y_pred = model.predict_classes(x)
    # y_pred = pd.DataFrame(y_pred)
    y_pred = y_pred.reshape(-1)
    data_test_submit = {'id': id_test, 'stroke': y_pred}
    df_test_submit = pd.DataFrame(data= data_test_submit)
    df_test_submit.to_csv('./Submission/submission_' + model_name + '.csv', index= False)

    return 

In [41]:
test_df = pd.read_csv("./Dataset/processed_test.csv")
generate_submission(model, test_df, 'ANN')

In [37]:
y_pred.reshape(-1)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,